**Función:** Entrenamiento de la CNN línea base con el dataset 10k balanceado.

**Requiere:**
- ../../datasets/X_train_DATASET10K224BAL_x_float.pkl
- ../../datasets/X_test_DATASET10K224BAL_x_float.pkl
- ../../datasets/y_train_DATASET10K224BAL.pkl
- ../../datasets/y_test_DATASET10K224BAL.pkl

**Crea:** Fichero con el modelo entrenado (epoch con menor val_loss).
- modelos/tf_model_\<run_id\>

In [ ]:
ejecutadoEnGoogleDrive = False # True si se ejecuta en Google Drive

In [ ]:
if ejecutadoEnGoogleDrive:
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = '/content/drive/clasificacion_imagenes/'
    ruta_datasets = '/content/drive/clasificacion_imagenes/datasets/'
else:
    PATH = '../../1_Setup_MLOps'
    ruta_datasets = '../../datasets/'

%cd {PATH}

In [ ]:
# !pip install mlflow
import mlflow
import numpy as np
from numpy.random import seed
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, Conv2D, MaxPooling2D
from tensorflow import keras
from tensorflow.random import set_seed
import time

In [2]:
# Fijar random state para obtener resultados reproductibles
seed(42) # So long and thanks for all the fish
set_seed(42)

In [ ]:
# Definir quién está ejecutando (para no machacar el MLflow de los compañeros)
# Descomentar la linea que aplique
# yo = 'Pedro_T'
# yo = 'Toni_V'
# yo = 'Carlos_H'
# yo = 'Carlos_C'
print('Notebook ejecutado por {}'.format(yo))

In [6]:
# Este run pertenence al experimento 2, que tiene el id "1" (string)
experiment_id = "1"
mlflow.set_experiment(experiment_id = experiment_id) 

<Experiment: artifact_location='mlruns', experiment_id='1', lifecycle_stage='active', name='Experimento 2 - evaluar impacto del balanceo', tags={'Dataset': 'DATASET 10K BAL 224',
 'Experimento': '2',
 'Modelo': 'CNN basica (baseline)'}>

In [ ]:
# OPCIÓN 1: crear run
Este texto garantiza que no creamos accidentalmente un nuevo run
run = mlflow.start_run(
  experiment_id=experiment_id,
  tags={"Autor": yo}
)
print('Vamos empezar el run {} en el experimento {}. Este run está {}.'.format(run.info.run_id, experiment_id, run.info.status))

In [7]:
# OPCIÓN 2: reanudar run
run_id = '6b6eff59143148e78dfe1a10efc7f71e' # por ej. "69bfdf296f384a75ada2ae55f9dc7672" (ver en la carpeta mlruns/0)
run = mlflow.start_run(run_id=run_id)
print('Vamos empezar el run {} en el experimento id_{}. Este run está {}.'.format(run.info.run_id, experiment_id, run.info.status))

Vamos empezar el run 6b6eff59143148e78dfe1a10efc7f71e en el experimento id_1. Este run está RUNNING.


In [8]:
# activar auto logging
mlflow.tensorflow.autolog()

Formato esperado:
- X_train: (N, 224, 224, 3) float32 (entre 0. y 1.)
- y_train: (N, 5) float32 (0. o 1.)
- X_test: (P, 224, 224, 3) float32 (entre 0. y 1.)
- y_test: (P, 5) float32 (0. o 1.)

In [11]:
X_train = pickle.load(open(PATH + 'X_train_DATASET10KBAL.pkl'.format(ruta_datasets),'rb'))
X_test = pickle.load(open(PATH + 'X_test_DATASET10KBAL.pkl'.format(ruta_datasets),'rb'))
y_train = pickle.load(open(PATH + 'y_train_DATASET10KBAL.pkl'.format(ruta_datasets),'rb'))
y_test = pickle.load(open(PATH + 'y_test_DATASET10KBAL.pkl'.format(ruta_datasets),'rb'))

In [12]:
categories = ['food', 'inside', 'outside', 'drink', 'menu']

In [13]:
print("X_train: {} {}".format(X_train.shape, X_train.dtype))
print("y_train: {} {}".format(y_train.shape, y_train.dtype))
print("X_test: {} {}".format(X_test.shape, X_test.dtype))
print("y_test: {} {}".format(y_test.shape, y_test.dtype))

X_train: (5500, 224, 224, 3) float32
y_train: (5500, 5) float32
X_test: (1375, 224, 224, 3) float32
y_test: (1375, 5) float32


In [15]:
# Dimensiones de la imagen de entrada
img_rows, img_cols, img_ch = 224, 224, 3
num_labels = 5

In [16]:
# test de los datos de entrada
if   (X_train.shape[1] != img_rows) \
  or (X_train.shape[2] != img_cols)\
  or (X_train.shape[3] != img_ch)\
  or (y_train.shape[1] != num_labels)\
  or (X_train.dtype != 'float32')\
  or (X_test.shape[1] != img_rows) \
  or (X_test.shape[2] != img_cols)\
  or (X_test.shape[3] != img_ch)\
  or (y_test.shape[1] != num_labels)\
  or (X_test.dtype != 'float32')\
  or (y_train.dtype != 'float32')\
  or (y_test.dtype != 'float32')\
  or (X_train.shape[0] != y_train.shape[0])\
  or (X_test.shape[0] != y_test.shape[0]\
  or type(X_train) != np.ndarray\
  or type(X_test) != np.ndarray\
  or type(y_train) != np.ndarray\
  or type(y_test) != np.ndarray):
  raise Exception("El dataset no tiene las dimensiones esperadas!")
else:
  print('El dataset tiene las dimensiones esperadas')

El dataset tiene las dimensiones esperadas


In [17]:
IMG_SIZE = 224
epochs = 40
batch_size=64
validation_split = 0.2

In [ ]:
model = Sequential()
model.add(Conv2D(10, (12, 12), padding='same', input_shape=(IMG_SIZE,IMG_SIZE,3)))
model.add(Activation('relu'))
model.add(Conv2D(IMG_SIZE, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.25))

model.add(Conv2D(IMG_SIZE, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(IMG_SIZE, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
opt = keras.optimizers.Nadam(learning_rate=0.0001, schedule_decay=1e-6)

In [19]:
# las métricas han sido quitadas de Keras en 2.0
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [20]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['acc', f1_m, precision_m, recall_m])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 10)      4330      
                                                                 
 activation (Activation)     (None, 224, 224, 10)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 222, 222, 224)     20384     
                                                                 
 activation_1 (Activation)   (None, 222, 222, 224)     0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 55, 55, 224)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 55, 55, 224)       0         
                                                        

In [22]:
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                         patience=8,
                                         restore_best_weights=True)

In [23]:
start_time = time.time()
history = model.fit(X_train, y_train,
                    epochs = epochs,
                    batch_size=batch_size,
                    validation_data=(X_test, y_test),
                    callbacks=[callback],
                    verbose=1)
end_time = time.time()

2022-09-17 09:57:31.417425: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3311616000 exceeds 10% of free system memory.
2022-09-17 09:57:52.888277: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2826141696 exceeds 10% of free system memory.
2022-09-17 09:58:26.640456: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2826141696 exceeds 10% of free system memory.
2022-09-17 09:58:38.717125: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2826141696 exceeds 10% of free system memory.
2022-09-17 09:58:47.275614: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2826141696 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: /tmp/tmpodz21d5g/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpodz21d5g/model/data/model/assets
2022/09/17 14:59:51 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpodz21d5g/model, flavor: keras), fall back to return ['tensorflow==2.9.1', 'keras==2.9.0']. Set logging level to DEBUG to see the full traceback.


In [30]:
model.save(PATH + 'modelos/tf_model_exp1_{}'.format(run.info.run_id))

INFO:tensorflow:Assets written to: Experimento_2/modelos/tf_model_exp1_6b6eff59143148e78dfe1a10efc7f71e/assets


INFO:tensorflow:Assets written to: Experimento_2/modelos/tf_model_exp1_6b6eff59143148e78dfe1a10efc7f71e/assets


In [31]:
mlflow.end_run()